<a href="https://colab.research.google.com/github/alex-movila/ML-Colab-Tutorials/blob/master/Keras_MNIST_DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [0]:
discriminator = keras.Sequential([
  keras.Input(shape=(28, 28, 1)),
  layers.Conv2D(64, (3, 3), strides=(2, 2), padding='same'),
  layers.LeakyReLU(alpha=0.2),
  layers.Conv2D(128, (3, 3), strides=(2, 2), padding='same'),
  layers.LeakyReLU(alpha=0.2),
  layers.GlobalMaxPooling2D(),
  layers.Dense(1)
], name='discriminator')

In [0]:
discriminator.summary()

Model: "discriminator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 14, 14, 64)        640       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 128)         73856     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 7, 7, 128)         0         
_________________________________________________________________
global_max_pooling2d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 74,625
Trainable params: 74,625
Non-trainable params: 0
_________________________________________________

In [0]:
latent_dim = 128

generator = keras.Sequential([
  keras.Input(shape=(latent_dim,)),
  # We want to generate 128 coefficients to reshape into a 7x7x128 map
  layers.Dense(7 * 7 * 128),
  layers.LeakyReLU(alpha=0.2),
  layers.Reshape((7, 7, 128)),
  layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'),
  layers.LeakyReLU(alpha=0.2),
  layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'),
  layers.LeakyReLU(alpha=0.2),
  layers.Conv2D(1, (7, 7), padding='same', activation='sigmoid')
], name='generator')

In [0]:
generator.summary()

Model: "generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 6272)              809088    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 6272)              0         
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 28, 28, 128)       262272    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 28, 28, 128)       0 

In [0]:
d_optimizer = keras.optimizers.Adam(learning_rate=0.0003)
g_optimizer = keras.optimizers.Adam(learning_rate=0.0004)

# Instantiate a loss function.
loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)

In [0]:
@tf.function
def train_step(real_images):
    # Sample random points in the latent space
    random_latent_vectors = tf.random.normal(shape=(batch_size, latent_dim))
    # Decode them to fake images
    generated_images = generator(random_latent_vectors)
    # Combine them with real images
    combined_images = tf.concat([generated_images, real_images], axis=0)

    # Assemble labels discriminating real from fake images
    labels = tf.concat([tf.ones((batch_size, 1)),
                        tf.zeros((real_images.shape[0], 1))], axis=0)
    # Add random noise to the labels - important trick!
    labels += 0.05 * tf.random.uniform(labels.shape)

    # Train the discriminator
    with tf.GradientTape() as tape:
      predictions = discriminator(combined_images)
      d_loss = loss_fn(labels, predictions)
    grads = tape.gradient(d_loss, discriminator.trainable_weights)
    d_optimizer.apply_gradients(zip(grads, discriminator.trainable_weights))

    # Sample random points in the latent space
    random_latent_vectors = tf.random.normal(shape=(batch_size, latent_dim))
    # Assemble labels that say "all real images"
    misleading_labels = tf.zeros((batch_size, 1))

    # Train the generator (note that we should *not* update the weights
    # of the discriminator)!
    with tf.GradientTape() as tape:
      predictions = discriminator(generator(random_latent_vectors))
      g_loss = loss_fn(misleading_labels, predictions)
    grads = tape.gradient(g_loss, generator.trainable_weights)
    g_optimizer.apply_gradients(zip(grads, generator.trainable_weights))
    return d_loss, g_loss, generated_images


In [0]:
# Prepare the dataset. We use both the training & test MNIST digits.
batch_size = 64
(x_train, _), (x_test, _) = keras.datasets.mnist.load_data()
all_digits = np.concatenate([x_train, x_test])
all_digits = all_digits.astype('float32') / 255.
all_digits = np.reshape(all_digits, (-1, 28, 28, 1))
dataset = tf.data.Dataset.from_tensor_slices(all_digits)
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)

In [0]:
import os
epochs = 100
save_dir = './'

for epoch in range(epochs):
  print('\nStart epoch', epoch)

  for step, real_images in enumerate(dataset):
    # Train the discriminator & generator on one batch of real images.
    d_loss, g_loss, generated_images = train_step(real_images)

    # Occasionally save / plot.
    if step % 200 == 0:
      # Print metrics
      print('discriminator loss at step %d: %.2f' % (step, d_loss))
      print('adversarial loss at step %d: %.2f' % (step, g_loss))

      # Save one generated image
      img = tf.keras.preprocessing.image.array_to_img(
          generated_images[0] * 255., scale=False)
      img.save(os.path.join(save_dir, 'generated_img' + str(step) + '.png'))


Start epoch 0
discriminator loss at step 0: 0.71
adversarial loss at step 0: 0.72
discriminator loss at step 200: 0.71
adversarial loss at step 200: 1.15
discriminator loss at step 400: 0.62
adversarial loss at step 400: 0.99
discriminator loss at step 600: 0.43
adversarial loss at step 600: 1.39
discriminator loss at step 800: 0.47
adversarial loss at step 800: 1.33
discriminator loss at step 1000: 0.67
adversarial loss at step 1000: 0.81

Start epoch 1
discriminator loss at step 0: 0.65
adversarial loss at step 0: 0.86
discriminator loss at step 200: 0.69
adversarial loss at step 200: 0.70
discriminator loss at step 400: 0.67
adversarial loss at step 400: 0.79
discriminator loss at step 600: 0.71
adversarial loss at step 600: 0.87
discriminator loss at step 800: 0.85
adversarial loss at step 800: 0.64
discriminator loss at step 1000: 0.72
adversarial loss at step 1000: 0.71

Start epoch 2
discriminator loss at step 0: 0.70
adversarial loss at step 0: 0.76
discriminator loss at step 